Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
SPDX-License-Identifier: Apache-2.0

# Exploring the UniProt protein knowledge base with AWS Open Data and Amazon Neptune

## Introduction 
The [Universal Protein Knowledge Base](https://www.uniprot.org/) (https://www.uniprot.org/) (UniProtKB) is a widely used protein data source that is now available through the Registry of Open Data on AWS. UniProt data is highly structured with many relationships between protein sequences, annotations, ontologies and other related data sources. UniProtKB can be directly accessed via the [UniProt website](https://www.uniprot.org/)  and is available for bulk downloads in several formats, including RDF which is particularly well suited to represent the complex and connected nature of the data as a graph. Creating a custom knowledge base can enable more advanced use cases, such as joining with other data sources, augmenting data with custom annotations and relationships, or inferring new relationships with analytics or machine learning.

In this example, we will demonstrate the step-by-step process to create and use your own protein knowledge base using UniProt RDF data. We will show how to ingest a subset of UniProtKB data into your own Amazon Neptune database directly from the Registry of Open Data on AWS. We will then show how to query the data with SPARQL, create new relationships in the data and visualise the data as a graph.

### UniProt Knowledge Base
UniProtKB is the central hub for the collection of functional information on proteins, with accurate, consistent and rich annotation. In addition to capturing the core data mandatory for each UniProtKB entry (mainly, the amino acid sequence, protein name, taxonomic data and citation information), as much additional annotation as possible is added. This includes widely accepted biological ontologies, classifications and cross-references, and clear indications of the quality of annotation in the form of evidence attribution of experimental and computational data.

### Registry of Open Data on AWS
The [Registry of Open Data](https://registry.opendata.aws/) on AWS makes it easy to find datasets made publicly available through AWS services. UniProtKB is made available through the Registry of Open Data via the [Open Data Sponsorship Program](https://aws.amazon.com/opendata/open-data-sponsorship-program/), which covers the cost of storage for publicly available high-value cloud-optimized datasets. UniProtKB is available in the registry as [RDF](https://www.w3.org/RDF/) files, a standard model for data interchange on the Web that is capable of capturing complex relationships within the data of the UniProtKB. 

> **Looking for other data sets or interested in publishing your data?** 
>
> [Browse available data](https://registry.opendata.aws/) and learn how to register your own datasets.

### Amazon Neptune
[Amazon Neptune](https://aws.amazon.com/neptune/) is a fast, reliable, fully managed graph database service that makes it easy to build and run applications that work with highly connected datasets. The core of Neptune is a purpose-built, high-performance graph database engine. This engine is optimized for storing billions of relationships and querying the graph with milliseconds latency. Neptune supports the popular graph query languages Apache TinkerPop Gremlin and W3C’s SPARQL, enabling you to build queries that efficiently navigate highly connected datasets.

### Creating a custom UniProt KB
In this example, we select a list of UniProt RDF files that we are interested in exploring. Then we ingest the RDF files from the Open Data Registry on AWS into Amazon Neptune DB. Once the data is ingested, we demonstrate how to query relationships and attributes. By adapting this example to your own research needs, you should be able to build a subset of UniProtKB containing data for your specific use case. This example will also be the foundation of our follow-on example where we will use [Neptune ML](https://docs.aws.amazon.com/neptune/latest/userguide/machine-learning.html) to predict relationships and attributes in the UniProtKB.

## Brief overview of UniProt data
UniProt RDF files are located in a public S3 bucket `s3://aws-open-data-uniprot-rdf`. There are different paths for each data release. New data releases typically happen every two months, so be sure to use the latest release directory. We use the first release of 2021, which has a path of `s3://aws-open-data-uniprot-rdf/2021-01`. For more information on the data, go to the [UniProt help pages](https://www.uniprot.org/help/). Here is a brief description for some of the different datasets and files.

UniProt KB data is located in a public S3 bucket `s3://aws-open-data-uniprot-rdf`.  There are different paths for each data release. New data releases typically happen every two months, so be sure to use the latest release directory. We will use a recent release for January, 2021, which has a path of  `s3://aws-open-data-uniprot-rdf/2021-01`. For more information on the data, go to the [UniProt help pages](https://www.uniprot.org/help/). Here is a brief description of some of the different data sets and files.

####  Gene Ontology (GO) and other reference data
These are the supporting datasets for UniProt that contain related information and meta data. Among these is the go.owl file. The [Web Ontology Language (OWL)](https://en.wikipedia.org/wiki/Web%5FOntology%5FLanguage) is used to define ontologies that describe taxonomies and classification networks, essentially defining the structure of the knowledge graph. OWL is also built upon RDF. UniProt uses the [Gene Ontology](http://geneontology.org/docs/introduction-to-go-resource/), which represents the current scientific knowledge about the functions of genes.


| Syntax      | Description |
| ----------- | ----------- |
| citations.rdf     | Literature citations       |
| diseases.rdf    | Human diseases        |
| journals.rdf     | Journals which contain articles cited in UniProt       |
| taxonomy.rdf   | Organisms        |
| keywords.rdf     | Keywords       |
| go.owl   | Gene Ontology        |
| enzyme.rdf      | Enzyme classification       |
| pathways.rdf   | Pathways        |
| locations.rdf     | Subcellular locations       |
| tissues.rdf  | Tissues        |
| databases.rdf    | Databases that are linked to from uniprot.rdf       |
| proteomes.rdf   | Proteomes        |

#### UniProt Sequence archive (UniParc)
The UniProt Archive (UniParc) is a comprehensive and non-redundant database that contains most of the publicly available protein sequences in the world. Data is partitioned into files of around 1 Gigabyte in size depending on the size of the protein sequence. For more information on this archive, see the [UniParc documentation](https://www.uniprot.org/help/uniparc).

#### UniProt Reference clusters (UniRef)
The UniProt Reference Clusters (UniRef) provide clustered sets of sequences from the UniProt Knowledgebase and selected UniParc records in order to obtain complete coverage of the sequence space at several resolutions while hiding redundant sequences. Unlike in UniParc, sequence fragments are merged in UniRef. The UniRef dataset is split into files that contain about 100,000 clusters each. For more information on reference clusters, see the [UniRef documentation](https://www.uniprot.org/help/uniref).

####  UniProt Knowledge Base (UniProt KB)
The UniProtKB dataset is split into files based on the top levels of the [NCBI taxonomy](https://www.ncbi.nlm.nih.gov/taxonomy) (the file name indicates the classification and ID of the taxon) that contain at most 1 million protein entries. Obsolete entries are provided in separate files with at most 10 million entries (uniprotkb_obsolete_\*.rdf). For more information on the knowledgebase, see the [UniProtKB documentation](https://www.uniprot.org/help/uniprotkb).

#### What does the data look like?
An RDF file is a document written in the Resource Description Framework (RDF) language, that was created to represent relationships between web resources. It’s also used to create ontologies for different domains. RDF contains information about an entity as structured metadata. RDF graphs contain statements with a subject, predicate and object, also known as a triple. The subject is the main actor, the predicate is the action or verb, and the object is what is acted upon. A triple can be used to associate a subject with a property or define a relationship between two subjects.

As an illustration, let’s look at how the taxonomy is represented in RDF. In the RDF listing, we see that [taxonomy 9606](https://www.uniprot.org/taxonomy/9606), is defined as Homo Sapiens. Taxonomy 9606 is a subclass of [taxonomy 9605](https://www.uniprot.org/taxonomy/9605), defined as Homininae. The subject is *taxonomy 9606*, the predicate is *subclass* and the object is *taxonomy 9605*. In addition, taxonomy 9606 has a narrower transitive relationship to [taxonomy 63221](https://www.uniprot.org/taxonomy/63221), defined as Homo sapiens neanderthalensis, as well as [taxonomy 741158](https://www.uniprot.org/taxonomy/741158), defined as Homo sapiens subsp. 'Denisova'. Expressed in RDF terms, the subject is *taxonomy 9606*, the predicate is *narrower transitive relationship* and the objects are *taxonomy 63221* and *taxonomy 741158*.

![image.png](./images/taxonRDF.png)

## Load the UniProt data
Instead of executing a large number of insert statements or other API calls, we use the [Amazon Neptune Bulk Loader](https://docs.aws.amazon.com/neptune/latest/userguide/bulk-load-data.html) to load a subset of UniProt RDF files directly from the public S3 bucket.

### Accessing the data
Before we can load data into the Neptune instance, we need an AWS Identity and Access Management (IAM) role that has access to the public bucket where the UniProt data resides. In addition, the Neptune loader requires a VPC endpoint for Amazon S3. For more information on bulk loading requirements, refer to the Neptune documentation on bulk loading](https://docs.aws.amazon.com/neptune/latest/userguide/bulk-load.html). The IAM role and VPC endpoint have already been configured by our cloud formation template. We just need to define the role and endpoint settings.

In [ ]:
import os
iamRoleArn = os.environ['NEPTUNE_LOAD_FROM_S3_ROLE_ARN']
neptune_host = os.environ['GRAPH_NOTEBOOK_HOST']
neptune_port = os.environ['GRAPH_NOTEBOOK_PORT']
url = f'https://{neptune_host}:{neptune_port}/loader'

### Determine the files to load
The entire UniProt dataset is over 10 Terabytes, and is growing rapidly. To save on time and cost, we only load the data that is relevant to our use case. Loading the complete UniProtKB will take around 74 hours, as the ingestion rate is around 6.6 GB per hour. The majority of loading cost is for the large DB writer instance. Using an r5.8xlarge, the cost per hour is 6.46 USD per hour, so the cost of writing the entire UniProt dataset would be around 478 USD in loading costs. For this example, we only load a single RDF file for Metazoa, the Gene Ontology file, and the taxonomy file. This keeps our total loading time to under one hour, and the cost below 10 USD.
These are the files to load:
- **taxonomy.rdf.gz** - the taxonomy file
- **go.rdf.gz** - the gene ontology file
- **uniprotkb_eukaryota_opisthokonta_metazoa_33208_0.rdf.gz** - a single UniProt KB file for Metazoa

In [ ]:
import json
import urllib.request

# define a list of files to be loaded
files_to_load = [
    "s3://aws-open-data-uniprot-rdf/2021-01/supporting/taxonomy.rdf.gz",
    "s3://aws-open-data-uniprot-rdf/2021-01/supporting/go.rdf.gz",
    "s3://aws-open-data-uniprot-rdf/2021-01/uniprot/uniprotkb_eukaryota_opisthokonta_metazoa_33208_0.rdf.gz"
]

### Execute the bulk load command
Now we have all the information needed to execute the bulk load command. In the code below we define a function that:
1.	Creates a json string of parameters required by the bulk loader command
2.	Creates and sends an http request to the bulk loader endpoint
3.	Returns bulk load tracking ids that will be used to track the loading process

We call this function for every file we wish to load. The entire loading process will take approximately 50 minutes. Times will vary depending on the region and the number and size of the datasets loaded.

Loading can also be accomplished with the %load command within the workbench. The load command works well for smaller files, but for loading large files, it makes it difficult to monitor the load process as it locks the entire notebook. 

In [ ]:
loadids = []

def loadfile(filelocation):
    data = {
      "source" : filelocation,
      "format" : "rdfxml",
      "iamRoleArn" : iamRoleArn,
      "region" : "eu-west-3",
      "failOnError" : "FALSE",
      "parallelism" : "OVERSUBSCRIBE",
      "queueRequest" : "TRUE"
    }

    data = json.dumps(data)

    req = urllib.request.Request(url = url, data = bytes(data.encode("utf-8")), method = "POST")

    req.add_header("Content-type", "application/json; charset=UTF-8")

    with urllib.request.urlopen(req) as resp:
        response_data = json.loads(resp.read().decode("utf-8"))
        loadId=response_data['payload']['loadId']
        
    print("load id: {}".format(loadId))
    loadids.append(loadId)
    return loadids
    
for file in files_to_load:
    loadids = loadfile(file)

### Monitor loading
Graph data comes in triples, which is the relationship between two nodes via an edge. Here are some metrics we captured during our previous load of the above files.

| File | File size  | triples | Approximate Load time in seconds |
| ----------- | ----------- | ----------- | ----------- |
| taxonomy.rdf.gz | 50.5MB   | 15950018 | 100 |
| go.rdf.gz | 3.3MB     | 381392  | 10 |
| uniprotkb_eukaryota_opisthokonta_metazoa_33208_0.rdf | 3.45GB   | 344412243 | 1900 |


Here is an estimation for a full load of all supporting files and UniProtKB data:

| File Directory | Number of Files | Total File Size  |  Approximate Load Time |
| ----------- | ----------- | ----------- |  ----------- |
| supporting/* |16   | 8.2 GB   | 1.25 hours |
| uniprot/*    | 275 | 482 GB   | 73 hours  |

Loading can take some time, so it's convenient to have a way to monitor the loading progress. The code below calls the Neptune endpoint with the load ID’s we saved previously, and returns a status update on the bulk loading process for each file.

In [ ]:
import time
for load_id in loadids:
    req = urllib.request.Request(url = "/".join([url, load_id]), method = "GET")
    req.add_header("Content-type", "application/json; charset=UTF-8")
    
    status = None
    while status != 'LOAD_COMPLETED':
        with urllib.request.urlopen(req) as resp:
            response_data = json.loads(resp.read().decode("utf-8"))['payload']

        status = response_data['overallStatus']['status']
        totalTimeSpent = response_data['overallStatus']['totalTimeSpent']
        totalRecords = response_data['overallStatus']['totalRecords']
        end = '\n' if status == 'LOAD_COMPLETED' else '\r'
        print(f"{load_id}  status: {status} \tload time: {totalTimeSpent}s\trecords: {totalRecords}", end=end)
        time.sleep(1)

### Downsize the Neptune instance

Once all the data is loaded, we no longer need a large writer instance, as we will mostly be reading from the database. It's recommended that you resize the Neptune database to a smaller instance after you finish loading the files to save on running costs.

From the Neptune Console, go to *Databases*, choose the Neptune Writer and then click *Modify*. 

![image.png](./images/step-1.png)

In the list of presented DB instance classes, select *db.r5.large*, the smallest instance available, and then click *Continue*.

![image.png](./images/step-2.png)
![image.png](./images/step-2a.png)

On the next page, choose *Apply immediately*, and click the *Modify DB Instance* button.

![image.png](./images/step-3.png)

Your Neptune DB instance is now cost optimized for querying. For more information, refer to the steps in the Neptune Developer Guide. For more information, refer the steps in the [Neptune Developer Guide](https://docs.aws.amazon.com/neptune/latest/userguide/manage-console-modify.html#manage-console-modify-instance).

## Querying the UniProtKB
Now that we have the loaded the UniProt data, we can query it with SPARQL, a query language for graph data in RDF format. Amazon Neptune is compatible with SPARQL 1.1. If you are used to standard SQL queries, working with SPARQL should be familiar to you. A SPARQL query contains:
- Prefixes to abbreviate URIs
- Dataset declaration to specify the graphs being queried
- SELECT clause that determines which attributes to return
- WHERE clause that specifies matching criteria
- Query modifiers for ordering results

If you are unfamiliar with SPARQL queries, see [Writing Simple Queries](https://www.w3.org/TR/sparql11-query/#WritingSimpleQueries) or the full guide in the [SPARQL 1.1 Query Language documentation](https://www.w3.org/TR/sparql11-query/).

Let's do some example queries, so you can get an idea of how SPARQL works with UniProtKB.

### Example 1:  Simple Query

Let's start with a simple query. In this query, we will look within the taxonomy tree, to see if there are any subclass records under *Homo Sapiens*. *Homo Sapiens* are coded with a taxonomy id of [9606](https://www.uniprot.org/taxonomy/9606). Lets query the web URI and scientific name of the subclass records. 

In [ ]:
%%sparql
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX up: <http://purl.uniprot.org/core/>
SELECT ?subject ?predicate ?object
WHERE
{
    ?subject a up:Taxon ;
               up:scientificName ?object ;
               rdfs:subClassOf taxon:9606 ;
               ?predicate ?object .
} 

The query returns two records, *Homo Sapiens Neanderthalensis* and *Homo Sapiens Subsp. 'Denisova'* , which are found under the taxonomy of Homo Sapiens. For taxonomy identifiers of other organisms, you may trace from the top node of [cellular organisms](https://www.uniprot.org/taxonomy/131567).

### Example 2: Query proteins and their related Gene Onotology (GO) code

Now lets find all the *Homo Sapiens* related preoteins that have a [Gene Onotlogy](http://geneontology.org) (GO) code. The Gene Ontology describes biological concepts with respect to molecular function, cellular components and biological processes. We will use the [9606](https://www.uniprot.org/taxonomy/9606) taxonomy code for *Homo Sapiens*. Since there are so many entries, we will also limit the query results to 50 entries. 

In the results returned, *`subject`* refers to the proteins, *`“object`* refers to the respective GO codings and *`predict`* are defined as the link as [classifiedWith](http://purl.uniprot.org/core/classifiedWith) from protein to GO. 

In [ ]:
%%sparql
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX up: <http://purl.uniprot.org/core/>
SELECT ?subject ?predicate ?object 
WHERE{ 
    ?object a owl:Class .
    ?subject a up:Protein ;       
               up:organism taxon:9606 ;
               up:classifiedWith ?object ;      
               ?predicate ?object .
}
LIMIT 50

The query returns the proteins URI along with the respective GO URI. Any proteins without a GO code will not be returned. We could also parse the GO URI to get the GO code if needed.

### Example 3: Filter proteins by GO code  
We can further filter the proteins with a specific GO code. For this query let’s visually inspect all the Homo Sapien proteins related to [GO_0005158](https://www.ebi.ac.uk/QuickGO/GTerm?id=GO:0005158), the GO code for insulin receptor binding. After we run the query, we can click on the "Graph" tab to see the related proteins in a graph diagram.

In [ ]:
%%sparql
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX go: <http://purl.obolibrary.org/obo/>
PREFIX up: <http://purl.uniprot.org/core/>
SELECT ?subject ?predicate ?object
WHERE{       
  ?subject a up:Protein ;
             up:classifiedWith go:GO_0005158 ;
             up:organism taxon:9606 ;
             up:classifiedWith ?object ;
             ?predicate ?object .
} 
Limit 50

### Example 4: Find ancestor GO codes

We can also query the ancestor or parent GO code of proteins. Ancestor GO codes are represented by subclass relationships from a GO code to its ancestor. For example, GO code [1900077](https://www.ebi.ac.uk/QuickGO/term/GO:1900077) is termed as the *“negative regulation of cellular response to insulin stimulus”* and has a *subClassOf* relation to the GO code [0048523](https://www.ebi.ac.uk/QuickGO/term/GO:0048523) which is termed the *“negative regulation of cell cycle”*. 

Since there is no direct link  between a protein and it ancestor GO code, we can only show a table of returned values, and can't visualize the results as a graph. We can also make it easier to read the results if we remove the full IRI by applying some string functions.


In [ ]:
%%sparql
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX go: <http://purl.obolibrary.org/obo/>
PREFIX up: <http://purl.uniprot.org/core/>

SELECT 
    (STRAFTER(STR(?protein), "uniprot/") AS ?protein_name)
    (STRAFTER(STR(?goCode), "obo/") AS ?go_code)
    (STRAFTER(STR(?superGoCode), "obo/") AS ?super_go_code)
WHERE{       
    ?protein a up:Protein ;
               up:classifiedWith go:GO_0005158 ;
               up:organism taxon:9606 ;
               up:classifiedWith ?goCode .
 
    ?goCode a owl:Class ;
              rdfs:subClassOf ?superGoCode .
    
   MINUS {
       ?protein up:classifiedWith ?superGoCode .
   }
} 
Limit 50

### Example 5:  Insert new shortcut links from protein to ancestor GO codes and visualize them

We were unable to visualize the graph in the example 4, since there was no direct link between a protein and its GO code’s ancestor. We can create that relationship by using the [`INSERT`](https://www.w3.org/TR/sparql11-update/#insertData) operation which adds selected triples into the graph. To do this, we  replace *`SELECT`* by *`INSERT`* ; all the selected triples are then added into the graph and you can then visualize them.

In [ ]:
%%sparql
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX go: <http://purl.obolibrary.org/obo/>
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX sc: <http://example.org/shortcuts/>

INSERT {
   ?subject sc:ancestorGo ?superGo
}
WHERE{
   ?subject a up:Protein ;
              up:classifiedWith go:GO_0005158 ;
              up:organism taxon:9606 ;
              up:classifiedWith ?object .
 
    ?object a owl:Class ;
              rdfs:subClassOf ?superGo .
   
   MINUS {
      ?subject up:classifiedWith ?superGo .
   }
} 

In [ ]:
%%sparql
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX go: <http://purl.obolibrary.org/obo/>
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX sc: <http://example.org/shortcuts/>

SELECT ?subject ?predicate ?object
WHERE{       
  ?subject a up:Protein ;
             up:classifiedWith go:GO_0005158 ;
             up:organism taxon:9606 ;
             sc:ancestorGo ?object ;
             ?predicate ?object .
} 
Limit 50

Once we have inserted the new shortcut relationships, we can now query them and look at the graph.

### Example 6: Adding data from Rhea to your Neptune Instance.
Rhea is an expert-curated knowledgebase of chemical and transport reactions of biological interest, and the standard for enzyme and transporter annotation in UniProtKB. Here in this example, we will use the public SPARQL endpoint of Rhea to enrich your Neptune instance by using the [INSERT](https://www.w3.org/TR/sparql11-update/#insertData) operation.

Starting with protein [Q9UBM7](https://www.uniprot.org/uniprot/Q9UBM7) which is a reductase that is part of the cholesterol biosynthetic pathway, we will demonstrate how to fetch the chemical names of the compounds reduced by the enzyme (i.e the chemical compounds of the enzyme-catalyzed reaction), and then insert the link between the protein in UniProtKB and the entrance in Rhea. 

The Rhea data is an external data store, so we use a [federated query](https://www.w3.org/TR/2013/REC-sparql11-federated-query-20130321/) with the [SERVICE](https://www.w3.org/TR/2013/REC-sparql11-federated-query-20130321/) keyword. As a best practice, we use a predicate prefix of *http://example.org/shortcuts* so we can distinguish the shortcut from the original UniProt data.

In [ ]:
%%sparql

PREFIX shortcut:<http://example.org/shortcuts/>
PREFIX rh:<http://rdf.rhea-db.org/>
PREFIX up:<http://purl.uniprot.org/core/>
INSERT {
    ?protein shortcut:metabolize ?chemical
}
WHERE {
    BIND(<http://purl.uniprot.org/uniprot/Q9UBM7> AS ?protein)
    
    ?protein up:reviewed true ;
             up:annotation ?a .
    
    ?a a up:Catalytic_Activity_Annotation ;
         up:catalyticActivity ?ca .
    
    ?ca up:catalyzedReaction ?reaction .
    
    SERVICE <https://sparql.rhea-db.org/sparql> {
        ?reaction rdfs:subClassOf rh:Reaction ;
                  rh:status rh:Approved ;                
                  rh:side ?reactionSide .
        ?reactionSide rh:contains ?participant .
        ?participant rh:compound ?compound .
        ?compound rh:chebi ?chemical .
    }
}

Once we have inserted the new shortcut relationships, the newly inserted triplets are ready for visualization. 

In [ ]:
%%sparql

PREFIX rh:<http://rdf.rhea-db.org/>
PREFIX up:<http://purl.uniprot.org/core/>
PREFIX shortcut:<http://example.org/shortcuts/>
        
SELECT 
   ?subject ?predicate ?object

WHERE
{   
    BIND(<http://purl.uniprot.org/uniprot/Q9UBM7> AS ?subject)
    
    ?subject up:reviewed true ;
             shortcut:metabolize ?object ;
             ?predicate ?object .
} 

# Conclusion
In this example notebook, we have demonstrated some simple ways to use your own protein KB using the UniProt data available on AWS Open Registry of Data and Amazon Neptune. If you would like to explore further, you can easily run this yourself in your own account. Some other things to try would be joining to other graph databases using federated queries, querying features  to train machine learning models, or inferring links with Neptune ML.


# **Danger Zone**

Use the following code to reset the database



In [ ]:
%db_reset